## Import packages

In [1]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import time
import pyomo.environ as pe
from pyomo.gdp import Disjunct, Disjunction
from pyomo.environ import ConstraintList, Block

## Model construction

### Creation of a Concrete Model

In [2]:
model = pe.ConcreteModel()

###  Define sets

#### Products

In [3]:
FEEDSTOCKS = ['hardwood', 'softwood', 'herbaceous-plant']
POLYMERS = ['cellulose','hemicellulose','lignin1']
PRODUCT1 = ['glucose','xylose','furfural','lignin2']
PRODUCT2 = ['lignin-monomers','char','unconverted-lignin']
PRODUCT3 = ['PHA','glycerol','methyl-oleate','adipic-acid','PDC']
FINALPRODUCT = ['PHA','methyl-oleate','adipic-acid','PDC']
BYPRODUCT = ['glucose','xylose','furfural','char','glycerol']
ALL_PRODUCT = FEEDSTOCKS + POLYMERS + PRODUCT1 + PRODUCT2 + PRODUCT3

#### Processes

In [4]:
MILLING = ['milling']
FRACTIONATION = ['DSA','SEP','LHW','organosolv','AFEX','GVL','alkaline']
DEPOLYMERIZATION = ['BCD','pyrolysis','HDO','HyThUp','HYDRO','noprocess','burn']
UPGRADING = ['PHA-ferm', 'lipid-ferm_biodiesel-prod', 'MA-ferm_AA-prod','PDC-ferm']
ALL_PROCESS = MILLING + FRACTIONATION + DEPOLYMERIZATION + UPGRADING

#### Chemicals and utility

In [5]:
CHEMICALS = ['water','NaOH','H2SO4','NH3','EtOH','anthraquinone','H2','cellulase-enzymes','CSL','DAP',
             'methanol','chloroform','H3PO4','GVL','glucose','AS','DSP','MPP','boiler_chem','FGD']
UTILITY =['cooling-water','electricity','natural-gas']

### Define parameters

#### Maximum input (ton/day)

In [6]:
max_inflow = 2000

#### Scaling coefficient

In [7]:
scal = 0.6

#### Feedstock composition

In [8]:
composition = pd.DataFrame(index = FEEDSTOCKS)

composition['cellulose'] = [0.46,0.38,0.49]
composition['hemicellulose'] = [0.28,0.32,0.36]
composition['lignin1'] = [0.26,0.3,0.15]

composition

,cellulose,hemicellulose,lignin1
hardwood,0.46,0.28,0.26
softwood,0.38,0.32,0.30
herbaceous-plant,0.49,0.36,0.15


#### Process relationships

In [9]:
process_rel = pd.DataFrame(index = ALL_PROCESS)

process_rel['milling'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['DSA'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['SEP'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['LHW'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['organosolv'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['AFEX'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['GVL'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['alkaline'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['BCD'] = [0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['pyrolysis'] = [0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['HDO'] = [0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['HyThUp'] = [0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['HYDRO'] = [0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0]
process_rel['noprocess'] = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
process_rel['burn'] = [0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
process_rel['PHA-ferm'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0]
process_rel['lipid-ferm_biodiesel-prod'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0]
process_rel['MA-ferm_AA-prod'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0]
process_rel['PDC-ferm'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0]
process_rel['PHA'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
process_rel['methyl-oleate'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
process_rel['adipic-acid'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
process_rel['PDC'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]

process_rel

,milling,DSA,SEP,LHW,organosolv,AFEX,GVL,alkaline,BCD,pyrolysis,...,noprocess,burn,PHA-ferm,lipid-ferm_biodiesel-prod,MA-ferm_AA-prod,PDC-ferm,PHA,methyl-oleate,adipic-acid,PDC
milling,0,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
DSA,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
SEP,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
LHW,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
organosolv,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
AFEX,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
GVL,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
alkaline,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
BCD,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0
pyrolysis,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0


#### Process yield

yield of products from fractionations

|  fractionation | polymer| fractionation product | yield |
|  :----:  | :----:  |:----:  |:----:  |
| DSA | cellulose| glucose | 0.91 |    
| DSA | hemicellulose| xylose | 0.90 |  
| DSA | lignin| lignin | 0.95 |
| DSA | lignin| lignin-monomers | 0 |
| SEP | cellulose| glucose | 0.68 |    
| SEP | hemicellulose| xylose | 0.58 | 
| SEP | lignin| lignin | 0.86 | 
| SEP | lignin| lignin-monomers | 0 | 
| LHW | cellulose| glucose | 1.02 |     
| LHW | hemicellulose| xylose | 0.78 | 
| LHW | lignin| lignin | 0.98 |  
| LHW | lignin| lignin-monomers | 0 |  
| organosolv | cellulose| glucose | 0.9 |    
| organosolv | hemicellulose| xylose | 0.81 |
| organosolv | lignin| lignin | 0.77 |
| organosolv | lignin| lignin-monomers | 0 |
| AFEX | cellulose| glucose | 0.90 |    
| AFEX | hemicellulose| xylose | 0.84 |  
| AFEX | lignin| lignin | 0.95 | 
| AFEX | lignin| lignin-monomers | 0 | 
| GVL | cellulose| glucose | 0.89 |    
| GVL | hemicellulose| furfural | 0.55 |  
| GVL | lignin| lignin | 0.72 | 
| GVL | lignin| lignin-monomers | 0 | 
| alkaline | cellulose| glucose | 0.37 |     
| alkaline | hemicellulose| xylose | 0.17 |  
| alkaline | lignin| lignin | 0 | 
| alkaline | lignin| lignin-monomers | 0.47 |

In [10]:
FRAC_PM = list((a,b) for a in FRACTIONATION for b in POLYMERS)
yield_fr = pd.DataFrame(index = FRAC_PM)

yield_fr['glucose'] = [0.91,0.0,0.0,0.68,0.0,0.0,1,0.0,0.0,0.9,0.0,0.0,0.9,0.0,0.0,0.89,0.0,0.0,0.37,0.0,0.0]
yield_fr['xylose'] = [0.0,0.9,0.0,0.0,0.58,0.0,0.0,0.78,0.0,0.0,0.81,0.0,0.0,0.84,0.0,0.0,0.0,0.0,0.0,0.17,0.0]
yield_fr['furfural'] = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.55,0.0,0.0,0.0,0.0]
yield_fr['lignin2'] = [0.0,0.0,0.95,0.0,0.0,0.86,0.0,0.0,0.98,0.0,0.0,0.77,0,0.0,0.95,0.0,0.0,0.72,0.0,0.0,0.47]

yield_fr

,glucose,xylose,furfural,lignin2
"(DSA, cellulose)",0.91,0.00,0.00,0.00
"(DSA, hemicellulose)",0.00,0.90,0.00,0.00
"(DSA, lignin1)",0.00,0.00,0.00,0.95
"(SEP, cellulose)",0.68,0.00,0.00,0.00
"(SEP, hemicellulose)",0.00,0.58,0.00,0.00
"(SEP, lignin1)",0.00,0.00,0.00,0.86
"(LHW, cellulose)",1.00,0.00,0.00,0.00
"(LHW, hemicellulose)",0.00,0.78,0.00,0.00
"(LHW, lignin1)",0.00,0.00,0.00,0.98
"(organosolv, cellulose)",0.90,0.00,0.00,0.00


yield of products from depolymerizations

In [11]:
yield_dpl = pd.DataFrame(index = DEPOLYMERIZATION)

yield_dpl['lignin-monomers'] = [0.36,0.1,0.25,0.24,0.23,1,0]
yield_dpl['char'] = [0,0.35,0,0.05,0,0,0]
yield_dpl['unconverted-lignin'] = [0.47,0,0.04,0.34,0.77,0,1]

yield_dpl

,lignin-monomers,char,unconverted-lignin
BCD,0.36,0.00,0.47
pyrolysis,0.10,0.35,0.00
HDO,0.25,0.00,0.04
HyThUp,0.24,0.05,0.34
HYDRO,0.23,0.00,0.77
noprocess,1.00,0.00,0.00
burn,0.00,0.00,1.00


In [12]:
yield_upg = pd.DataFrame(index = UPGRADING)

yield_upg['PHA']=[0.13,0,0,0]
yield_upg['glycerol']=[0,0.1*0.17,0,0]
yield_upg['methyl-oleate']=[0,0.97*0.17,0,0]
yield_upg['adipic-acid']=[0,0,0.66,0]
yield_upg['PDC']=[0,0,0,0.89]

yield_upg

,PHA,glycerol,methyl-oleate,adipic-acid,PDC
PHA-ferm,0.13,0.000,0.0000,0.00,0.00
lipid-ferm_biodiesel-prod,0.00,0.017,0.1649,0.00,0.00
MA-ferm_AA-prod,0.00,0.000,0.0000,0.66,0.00
PDC-ferm,0.00,0.000,0.0000,0.00,0.89


#### Process conditions

Chemicals usage of all processes

In [13]:
ch = pd.DataFrame(index = ALL_PROCESS)

ch['water'] = [0,4,4,9,4,4,4,4,4,0,0,5.06,0,0,0,6.7,9+0.17*4,11.5,9]
ch['NaOH'] = [0,0,0,0,0,0,0,0.04,0.1,0,0,1,0,0,0,0,0.17*0.01,0,0]
ch['H2SO4'] = [0,0.009,0,0,0.0175,0,0.0392,0.049,0.0012,0,0,0.012,0,0,0,0,0,0,0]
ch['NH3'] = [0,0.003,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
ch['EtOH'] = [0,0,0,0,0.032,0,0,0,0,0,0,0,0,0,0,0,0,0.66*0.0034,0]
ch['anthraquinone'] = [0,0,0,0,0,0,0,0.0005,0,0,0,0,0,0,0,0,0,0,0]
ch['H2'] = [0,0,0,0,0,0,0,0,0,0,0.027,0,0,0.025,0,0,0,0.66*0.0366,0]
ch['cellulase-enzymes'] = [0,0.01,0.01,0.01,0.01,0.01,0,0.01,0,0,0,0,0,0,0,0,0,0,0]
ch['CSL'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.3,0.35,0.2,0]
ch['DAP'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.01,0.1,0.1,0]
ch['methanol'] = [0,0,0,0,0,0,0,0,0,0,0,0,0.051,0,0,0,0.17*0.2,0,0]
ch['chloroform'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.17*0.02,0,0]
ch['H3PO4'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.17*0.0085,0,0]
ch['GVL'] = [0,0,0,0,0,0,2.8,0,0,0,0,0,0,0,0,0,0,0,0]
ch['glucose'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1]
ch['AS'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.05]
ch['DSP'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.14]
ch['MPP'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.14]
ch['boiler_chem'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
ch['FGD'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

ch

,water,NaOH,H2SO4,NH3,EtOH,anthraquinone,H2,cellulase-enzymes,CSL,DAP,methanol,chloroform,H3PO4,GVL,glucose,AS,DSP,MPP,boiler_chem,FGD
milling,0.00,0.0000,0.0000,0.000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
DSA,4.00,0.0000,0.0090,0.003,0.000000,0.0000,0.000000,0.01,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
SEP,4.00,0.0000,0.0000,0.000,0.000000,0.0000,0.000000,0.01,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
LHW,9.00,0.0000,0.0000,0.000,0.000000,0.0000,0.000000,0.01,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
organosolv,4.00,0.0000,0.0175,0.000,0.032000,0.0000,0.000000,0.01,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
AFEX,4.00,0.0000,0.0000,1.000,0.000000,0.0000,0.000000,0.01,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
GVL,4.00,0.0000,0.0392,0.000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000,0.0000,0.000000,2.8,0.0,0.00,0.00,0.00,0,0
alkaline,4.00,0.0400,0.0490,0.000,0.000000,0.0005,0.000000,0.01,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
BCD,4.00,0.1000,0.0012,0.000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0
pyrolysis,0.00,0.0000,0.0000,0.000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000,0.0000,0.000000,0.0,0.0,0.00,0.00,0.00,0,0


Utility usage of all processes

In [14]:
ut = pd.DataFrame(index = ALL_PROCESS)

ut['electricity'] = [200,1.19,0.53,1.87,184.13,239.25,175.44,1.18,1.18,14.12,5.6,19.8,48,0,0,352.22,42.75+0.17*0.8,325.13,909.35]
ut['cooling-water'] = [0,633.33,819.44,866.67,2.18,1491.67,6922.8,100,100,8.96,8.36,24.92,3328.54,0,0,9.7,2228.23+0.17*906.35,24899.71,3854.7]
ut['natural-gas'] = [0,544.43,525.05,691.65,595.24,586.08,6433.2,289.16,289.16,0,72,1260,2022.6,0,0,503.16,1423.46+0.17*882.35,12195.47,5879.04]

ut

,electricity,cooling-water,natural-gas
milling,200.000,0.0000,0.0000
DSA,1.190,633.3300,544.4300
SEP,0.530,819.4400,525.0500
LHW,1.870,866.6700,691.6500
organosolv,184.130,2.1800,595.2400
AFEX,239.250,1491.6700,586.0800
GVL,175.440,6922.8000,6433.2000
alkaline,1.180,100.0000,289.1600
BCD,1.180,100.0000,289.1600
pyrolysis,14.120,8.9600,0.0000


#### Product and chemical price

In [15]:
# price of feedstock ($/MT)

price_fs = pd.DataFrame(index = FEEDSTOCKS)
price_fs['price'] = [78.0,87.0,70.0]
price_fs

,price
hardwood,78.0
softwood,87.0
herbaceous-plant,70.0


In [16]:
# price of chemicals ($/MT)

price_ch = pd.DataFrame(index = CHEMICALS)
price_ch['price'] = [3.75,580.4,104.69,462.57,820.46,4643.46,1778.69,1191.97,83.56,387,404.25,590,750,
                     1077.18,893.49,292.02,1039.21,935.29,6601.39,263.57]
price_ch

,price
water,3.75
NaOH,580.40
H2SO4,104.69
NH3,462.57
EtOH,820.46
anthraquinone,4643.46
H2,1778.69
cellulase-enzymes,1191.97
CSL,83.56
DAP,387.00


In [17]:
# price of utility ($/kWh)

price_ut = pd.DataFrame(index = UTILITY)
price_ut['price'] = [0.000795,0.137,0.02]
price_ut

,price
cooling-water,0.000795
electricity,0.137000
natural-gas,0.020000


In [18]:
# price of final products ($/MT)

price_p3 = pd.DataFrame(index = FINALPRODUCT)
price_p3['price'] = [6300.0,1250,2190.0,5500.0]

price_p3

,price
PHA,6300.0
methyl-oleate,1250.0
adipic-acid,2190.0
PDC,5500.0


In [19]:
# price of byproducts ($/MT)

price_by = pd.DataFrame(index = BYPRODUCT)
price_by['price'] = [464.82,464.82,1077.18,118,1118.27]
price_by

,price
glucose,464.82
xylose,464.82
furfural,1077.18
char,118.00
glycerol,1118.27


#### Process base size and CAPEX

In [20]:
base = pd.DataFrame(index = ALL_PROCESS)

base['base_size'] = [2000,2000,2481,2000,2124,2000,2000,640,640,600,600,600,400,1,1,75.82,1207,633,84.72]
base['base_capex'] = [41.44,145.38,153.34,144.26,136.93,183.4,386.3,58.19,16.93,69.24,84.01,
                      94.17,17.03,0,0,27.59,41.9+3.2*(1207/(181/0.17))**scal,141.81,35.78]

base

,base_size,base_capex
milling,2000.00,41.440000
DSA,2000.00,145.380000
SEP,2481.00,153.340000
LHW,2000.00,144.260000
organosolv,2124.00,136.930000
AFEX,2000.00,183.400000
GVL,2000.00,386.300000
alkaline,640.00,58.190000
BCD,640.00,16.930000
pyrolysis,600.00,69.240000


#### Economic factors

In [21]:
n = 20 # operating year
ir = 0.1 # discount rate
tax = 0.21 # tax rate
days = 330 # operating days for a year
work_hour = 7920 # working hours per year
ft = [0.08,0.6,0.32]+[0.0]*27 # fraction of capital expenditure spent at year t

### Define variables

#### Selection of feedstock and processes

In [22]:
# Binary variable for If(fs) and Ip(all process)
model.Ifs = pe.Var(FEEDSTOCKS, domain=pe.Boolean, doc='selection of feedstock')
model.Ip = pe.Var(ALL_PROCESS, domain=pe.Boolean, doc='selection of process')

#### Product flow 

In [23]:
# All processes
model.Fin = pe.Var(ALL_PROCESS, domain=pe.NonNegativeReals, doc='inflow')
model.Fout = pe.Var(ALL_PROCESS, domain=pe.NonNegativeReals, doc='outflow')

# Linkage
model.F = pe.Var(ALL_PROCESS, ALL_PROCESS, ALL_PRODUCT, domain=pe.NonNegativeReals, doc='linkage',initialize=0)

In [24]:
# milling inflow and outflow
model.Finmill = pe.Var(MILLING,POLYMERS,domain=pe.NonNegativeReals,doc='inflow of milling',initialize=0)
model.Foutmill = pe.Var(MILLING,POLYMERS,domain=pe.NonNegativeReals,doc='outflow of milling',initialize=0)

In [25]:
# Fractionation inflow and outflow
model.Finfr = pe.Var(FRACTIONATION,POLYMERS,domain=pe.NonNegativeReals,doc='inflow of fractionation',initialize=0)
model.Foutfr = pe.Var(FRACTIONATION,PRODUCT1,domain=pe.NonNegativeReals,doc='outflow of fractionation',initialize=0)

# Fractionation unconverted part used to burn
model.Fburnfr = pe.Var(FRACTIONATION,POLYMERS,domain=pe.NonNegativeReals,doc='unconverted part of fractionation',initialize=0)

In [26]:
# Depolymerization outflow (more than one products)
model.Findpl = pe.Var(DEPOLYMERIZATION,['lignin2'],domain=pe.NonNegativeReals,doc='inflow of depolymerization',initialize=0)
model.Foutdpl = pe.Var(DEPOLYMERIZATION,PRODUCT2,domain=pe.NonNegativeReals,doc='outflow of depolymerization',initialize=0)

In [27]:
# upgrading outflow (more than one products)
model.Finupg = pe.Var(UPGRADING,['lignin-monomers'],domain=pe.NonNegativeReals,doc='inflow of upgrading',initialize=0)
model.Foutupg = pe.Var(UPGRADING,PRODUCT3,domain=pe.NonNegativeReals,doc='outflow of upgrading',initialize=0)

#### Chemicals and utility flow

In [28]:
# chemicals
model.chin = pe.Var(ALL_PROCESS,CHEMICALS,domain=pe.NonNegativeReals,doc='chemical of processes')

# utility
model.utin = pe.Var(ALL_PROCESS,UTILITY,domain=pe.NonNegativeReals,doc='outflow of processes')

### Size fraction

In [29]:
model.sizefraction = pe.Var(ALL_PROCESS,domain = pe.NonNegativeReals,doc = 'size fraction')

### Define constraints

#### Process selection

In [30]:
def one_feedstock_rule(model, i):
    if i == 'softwood':
        return model.Ifs[i] == 1
    else:
        return model.Ifs[i] == 0
model.onefeedstock = pe.Constraint(FEEDSTOCKS, rule=one_feedstock_rule)

In [31]:
def one_milling_rule(model,i):
    return model.Ip[i] == 1
model.onemilling = pe.Constraint(MILLING,rule=one_milling_rule)

In [32]:
# Select only one fractionation
def one_frac_rule(model, i):
    return sum(model.Ip[i] for i in FRACTIONATION) == 1
model.onefrac = pe.Constraint(FRACTIONATION, rule=one_frac_rule, doc='Select only one fractionation')

In [33]:
# Select only one depolymerization
def one_dpl_rule(model, i):
    return sum(model.Ip[i] for i in DEPOLYMERIZATION) == 1
model.onedpl = pe.Constraint(DEPOLYMERIZATION, rule=one_dpl_rule, doc='Select only one depolymerization')

In [34]:
# Select only one upgrading1
def one_upg_rule(model, i):
    return sum(model.Ip[i] for i in UPGRADING) <= 1
model.oneupg = pe.Constraint(UPGRADING, rule=one_upg_rule, doc='Select only one upgrading')

In [35]:
# inflow constraint
def inflow_constraint_rule0(model, i):
    return model.Ip[i] <= sum(process_rel[i][j]* model.Ip[j] for j in MILLING)
model.in0constraint = pe.Constraint(FRACTIONATION, rule=inflow_constraint_rule0)

def inflow_constraint_rule1(model, i):
    return model.Ip[i] <= sum(process_rel[i][j]* model.Ip[j] for j in FRACTIONATION)
model.in1constraint = pe.Constraint(DEPOLYMERIZATION, rule=inflow_constraint_rule1)

def inflow_constraint_rule2(model, i):
    return model.Ip[i] <= sum(process_rel[i][j]* model.Ip[j] for j in DEPOLYMERIZATION)
model.in2constraint = pe.Constraint(UPGRADING, rule=inflow_constraint_rule2)

In [36]:
# outflow constraint
def outflow_constraint_rule0(model, i):
    return model.Ip[i] <= sum(process_rel[j][i]* model.Ip[j] for j in FRACTIONATION)
model.out0constraint = pe.Constraint(MILLING, rule=outflow_constraint_rule0)

def outflow_constraint_rule1(model, i):
    return model.Ip[i] <= sum(process_rel[j][i]* model.Ip[j] for j in DEPOLYMERIZATION)
model.out1constraint = pe.Constraint(FRACTIONATION, rule=outflow_constraint_rule1)

def outflow_constraint_rule2(model, i):
    return model.Ip[i] <= sum(process_rel[j][i]* model.Ip[j] for j in UPGRADING)
model.out2constraint = pe.Constraint(DEPOLYMERIZATION, rule=outflow_constraint_rule2)

#### Process product flow

##### Milling

In [37]:
# milling inflow of polymers
def milling_polymer_inflow_rule(model,i,n):
    return model.Finmill[i,n] == sum(model.Ifs[m] * max_inflow * composition[n][m] for m in FEEDSTOCKS)
model.milling_polymer_inflow = pe.Constraint(MILLING, POLYMERS, rule=milling_polymer_inflow_rule, doc='milling polymer inflow')

# milling total inflow 
def milling_inflow_rule(model,i):
    return model.Fin[i] == sum(model.Finmill[i,n] for n in POLYMERS)
model.millinginflow = pe.Constraint(MILLING, rule=milling_inflow_rule, doc='milling total inflow')

In [38]:
# milling outflow of polymers
def milling_polymer_outflow_rule(model,i,n):
    return model.Foutmill[i,n] == model.Finmill[i,n]
model.milling_polymer_outflow = pe.Constraint(MILLING, POLYMERS, rule=milling_polymer_outflow_rule, doc='milling polymer outflow')

# milling outflow
def milling_outflow_rule(model,i):
    return  model.Fout[i] == model.Fin[i]
model.millingoutflow = pe.Constraint(MILLING, rule=milling_outflow_rule, doc='milling total outflow')

In [39]:
# linkage of milling and fractionation
def milling_fractionation_rule(model):
    for i in MILLING:
        for j in FRACTIONATION:
            for n in POLYMERS:
                yield model.F[i,j,n] <= model.Foutmill[i,n] * process_rel[j][i]
                yield model.Foutmill[i,n] == sum(model.F[i,j,n] for j in FRACTIONATION)
model.milling_fractionation = pe.ConstraintList(rule=milling_fractionation_rule)

##### Fractionation

In [40]:
# Fractionation polymer inflow
def frac_polymer_inflow_rule(model,i,n):
    return  model.Finfr[i,n] == sum(model.F[j,i,n] for j in MILLING)
model.frac_polymer_inflow = pe.Constraint(FRACTIONATION, POLYMERS, rule=frac_polymer_inflow_rule, doc='Fractionation polymer inflow')

# Fractionation total inflow
def frac_inflow_rule(model,i):
    return  model.Fin[i] == sum(model.Finfr[i,n] for n in POLYMERS)
model.fracinflow = pe.Constraint(FRACTIONATION, rule=frac_inflow_rule, doc='Fractionation total inflow')

In [41]:
# big M formulation

M = 10*max_inflow

def bigM_fr(model):
    for i in FRACTIONATION:
        for j in MILLING:
            for n in POLYMERS:
                yield model.F[j,i,n] - M * model.Ip[i] <= 0
                yield model.F[j,i,n] >= 0
                
model.bigM_fr = pe.ConstraintList(rule=bigM_fr)

In [42]:
# Fractionation Reaction
def frac_reaction_rule(model,i,n):
    return sum(model.Finfr[i,m] * yield_fr[n][i,m] for m in POLYMERS) == model.Foutfr[i,n]
model.fracreaction = pe.Constraint(FRACTIONATION,PRODUCT1,rule=frac_reaction_rule, doc='Fractionation reaction')

In [43]:
# Unconverted carbohydrates flow to boiler/turbogenerator
def frac_burn_rule(model,i,m):
    return model.Finfr[i,m] * (1 - sum(yield_fr[n][i,m] for n in PRODUCT1)) == model.Fburnfr[i,m]
model.fracunconvert = pe.Constraint(FRACTIONATION,POLYMERS,rule=frac_burn_rule, doc='Fractionation unconverted')

In [44]:
# Fractionation outflow to next stage (lignin)
def frac_outflow_rule(model,i):
    return  model.Fout[i] == model.Foutfr[i,'lignin2'] 
model.fracoutflow = pe.Constraint(FRACTIONATION, rule=frac_outflow_rule, doc='Fractionation outflow to next stage')

In [45]:
# linkage of fractionation and depolymerization
def fractionation_depolymerization_rule(model):
    for i in FRACTIONATION:
        for j in DEPOLYMERIZATION:
            for n in ['lignin2']:
                yield model.F[i,j,n] <= model.Foutfr[i,n] * process_rel[j][i]
                yield model.Foutfr[i,n] == sum(model.F[i,j,n] for j in DEPOLYMERIZATION)
model.fractionation_depolymerization = pe.ConstraintList(rule=fractionation_depolymerization_rule)

##### Depolymerization

In [46]:
# Depolymerization lignin inflow
def dpl_lignin_inflow_rule(model,i,n):
    return  model.Findpl[i,n] == sum(model.F[j,i,n] for j in FRACTIONATION)
model.dpl_lignin_inflow = pe.Constraint(DEPOLYMERIZATION, ['lignin2'], rule=dpl_lignin_inflow_rule)

# Depolymerization total inflow
def dpl_inflow_rule(model,i):
    return  model.Fin[i] == sum(model.Findpl[i,n] for n in ['lignin2'])
model.dplinflow = pe.Constraint(DEPOLYMERIZATION, rule=dpl_inflow_rule, doc='Depolymerization total inflow')

In [47]:
# big M formulation

M = 5*max_inflow

def bigM_dpl(model):
    for i in DEPOLYMERIZATION:
        for j in FRACTIONATION:
            for n in ['lignin2']:
                yield model.F[j,i,n] - M * model.Ip[i] <= 0
                yield model.F[j,i,n] >= 0

model.bigM_dpl = pe.ConstraintList(rule=bigM_dpl)

In [48]:
# Depolymerization Reaction
def dpl_reaction_rule(model,i,n):
    return model.Foutdpl[i,n] == model.Findpl[i,'lignin2'] * yield_dpl[n][i]
model.dplreaction = pe.Constraint(DEPOLYMERIZATION,PRODUCT2,rule=dpl_reaction_rule, doc='Depolymerization reaction')

In [49]:
# Depolymerization outflow to next stage (lignin-monomers)
def dpl_outflow_rule(model,i):
    return  model.Fout[i] == model.Foutdpl[i,'lignin-monomers']
model.dploutflow = pe.Constraint(DEPOLYMERIZATION, rule=dpl_outflow_rule, doc='Depolymerization outflow to next stage')

In [50]:
# linkage of depolymerization and upgrading
def depolymerization_upgrading_rule(model):
    for i in DEPOLYMERIZATION:
        for j in UPGRADING:
            for n in ['lignin-monomers']:
                yield model.F[i,j,n] <= model.Foutdpl[i,n] * process_rel[j][i]
                yield model.Foutdpl[i,n] == sum(model.F[i,j,n] for j in UPGRADING)
model.depolymerization_upgrading = pe.ConstraintList(rule=depolymerization_upgrading_rule)

##### Upgrading

In [51]:
# upgrading lignin monomer inflow
def upg_monomer_inflow_rule(model,i,n):
    return model.Finupg[i,n] == sum(model.F[j,i,n] for j in DEPOLYMERIZATION)
model.upg_monomer_inflow = pe.Constraint(UPGRADING,['lignin-monomers'], rule=upg_monomer_inflow_rule)

# upgrading inflow
def upg_inflow_rule(model, i):
    return model.Fin[i] == sum(model.Finupg[i,n] for n in ['lignin-monomers'])
model.upginflow = pe.Constraint(UPGRADING, rule=upg_inflow_rule, doc='upgrading inflow')

In [52]:
# big M formulation

M = 3*max_inflow

def bigM_upg(model):
    for i in UPGRADING:
        for j in DEPOLYMERIZATION:
            for n in ['lignin-monomers']:
                yield model.F[j,i,n] - M * model.Ip[i] <= 0
                yield model.F[j,i,n] >= 0

model.bigM_upg = pe.ConstraintList(rule=bigM_upg)

In [53]:
# upgrading Reaction
def upg_reaction_rule(model,i,n):
    return model.Foutupg[i,n] == model.Finupg[i,'lignin-monomers'] * yield_upg[n][i]
model.upgreaction = pe.Constraint(UPGRADING,PRODUCT3,rule=upg_reaction_rule, doc='upgrading reaction')

In [54]:
# upgrading to final bioproducts
def upg_outflow_rule(model,i):
    return  model.Fout[i] == sum(model.Foutupg[i,n] for n in FINALPRODUCT) 
model.upgoutflow = pe.Constraint(UPGRADING, rule=upg_outflow_rule, doc='upgrading to final products')

#### Size fraction

In [55]:
def size_fraction_rule(model,i):
    return model.sizefraction[i] == model.Fin[i]/base['base_size'][i]
model.fractionrule = pe.Constraint(ALL_PROCESS,rule = size_fraction_rule, doc = 'Size fraction')

#### Chemical and utility flow

In [56]:
# chemical
def chemical_rule(model, i, m):
    if i in FRACTIONATION and m == 'cellulase-enzymes':
        return model.chin[i,m] == model.Finfr[i,'cellulose'] * ch[m][i]
    else:
        return model.chin[i,m] == model.Fin[i] * ch[m][i]
model.chflow = pe.Constraint(ALL_PROCESS, CHEMICALS, rule=chemical_rule, doc='Chemicals flow of process')

In [57]:
# utility
def utility_rule(model, i, m):
    return model.utin[i,m] == model.Fin[i] * ut[m][i]
model.utflow = pe.Constraint(ALL_PROCESS,UTILITY, rule=utility_rule, doc='Utility flow of process')

## Objective

### Boiler/Turbogenerator

In [58]:
# Lower heating value of lignin and carbonhydrates (unit: MWh/MT)

LHV_lignin = 5.6
LHV_carhyd = 4.76

# boiler/turbogenerator power generation efficiency
eff = 0.43 

In [59]:
# Boiler/Turbogenerator inflow (energy)
Fin_BOTU = LHV_lignin * sum(model.Foutdpl[j,'unconverted-lignin'] for j in DEPOLYMERIZATION)\
         + LHV_lignin * sum(model.Fburnfr[j,'lignin1'] for j in FRACTIONATION)\
         + LHV_carhyd * sum(model.Fburnfr[j,'cellulose'] for j in FRACTIONATION)\
         + LHV_carhyd * sum(model.Fburnfr[j,'hemicellulose'] for j in FRACTIONATION)
         
# Boiler/Turbogenerator outflow (power generation, unit: kWh)
Fout_BOTU = eff * Fin_BOTU * 1000

In [60]:
# Equipment cost
base_size_BOTU = 2627
base_EC_BOTU = 90.63

EC_BOTU = base_EC_BOTU * (Fin_BOTU/base_size_BOTU)**scal

In [61]:
# Chemical cost
boiler_chem_BOTU = 1.83e-6
FGD_BOTU = 1.64e-3

ch_BOTU = Fin_BOTU * (boiler_chem_BOTU * price_ch.loc['boiler_chem'].item() + FGD_BOTU * price_ch.loc['FGD'].item())

### Waste Treatment

In [62]:
# equipment cost
base_size_w = 2000
base_EC_w = 41.59

EC_w = base_EC_w*(max_inflow/base_size_w)**scal

In [63]:
# estimated waste treatment operating cost
base_size_w = 2000
base_OPEX_w = 1.69

Cw = base_OPEX_w/base_size_w*max_inflow*1e6/days

### CAPEX

In [64]:
# The calculation is based on NREL and Ke Wang 2021

# Equipment cost
EC = sum((base['base_capex'][i] * (model.sizefraction[i])**scal) for i in ALL_PROCESS) \
   + EC_w + EC_BOTU

In [65]:
# Estimated capital cost
CAPEX = 3.85*EC

In [66]:
# used for result analysis

# Fractionation Equipment cost
EC1 = sum(model.Ip[i] * (base['base_capex'][i] * (model.sizefraction[i])**scal) for i in FRACTIONATION)

# Estimated capital cost
CAPEX1 = 3.85*EC

# depolymerization Equipment cost
EC2 = sum(model.Ip[i] * (base['base_capex'][i] * (model.sizefraction[i])**scal) for i in DEPOLYMERIZATION)

# Estimated capital cost
CAPEX2 = 3.85*EC

# upgrading Equipment cost
EC3 = sum(model.Ip[i] * (base['base_capex'][i] * (model.sizefraction[i])**scal) for i in UPGRADING)

# Estimated capital cost
CAPEX3 = 3.85*EC

### OPEX

In [67]:
chemical_cost = sum(model.chin[i,m] * price_ch.loc[m].item() for i in ALL_PROCESS for m in CHEMICALS)\
              + ch_BOTU
utility_cost = sum(model.utin[i,m] * price_ut.loc[m].item() for i in ALL_PROCESS for m in UTILITY)

In [68]:
# fixed operating cost
FOC = CAPEX*1000000 *0.03/days

OPEX= chemical_cost + utility_cost + Cw + FOC

feedstock_cost = sum(model.Ifs[m] * max_inflow * price_fs.loc[m].item() for m in FEEDSTOCKS)

In [69]:
chemical_cost1 = sum(model.Ip[i] * model.chin[i,m] * price_ch.loc[m].item() for i in FRACTIONATION for m in CHEMICALS)
utility_cost1 = sum(model.Ip[i] * model.utin[i,m] * price_ut.loc[m].item() for i in FRACTIONATION for m in UTILITY)

chemical_cost2 = sum(model.Ip[i] * model.chin[i,m] * price_ch.loc[m].item() for i in DEPOLYMERIZATION for m in CHEMICALS)
utility_cost2 = sum(model.Ip[i] * model.utin[i,m] * price_ut.loc[m].item() for i in DEPOLYMERIZATION for m in UTILITY)

chemical_cost3 = sum(model.Ip[i] * model.chin[i,m] * price_ch.loc[m].item() for i in UPGRADING for m in CHEMICALS)
utility_cost3 = sum(model.Ip[i] * model.utin[i,m] * price_ut.loc[m].item() for i in UPGRADING for m in UTILITY)

FOC1 = CAPEX1*1000000 *0.03/days
FOC2 = CAPEX2*1000000 *0.03/days
FOC3 = CAPEX3*1000000 *0.03/days

OPEX1= chemical_cost1 + utility_cost1 + FOC1
OPEX2= chemical_cost2 + utility_cost2 + FOC2
OPEX3= chemical_cost3 + utility_cost3 + FOC3

#### utility usage (result analysis)

In [70]:
# daily utility usage
# UTILITY =['cooling-water','electricity','natural-gas']
electrcity_all = sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['electricity'])
electrcity_1 = sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['electricity'])
electrcity_2 = sum(model.Ip[i] * model.utin[i,m] for i in DEPOLYMERIZATION for m in ['electricity'])
electrcity_3 = sum(model.Ip[i] * model.utin[i,m] for i in UPGRADING for m in ['electricity'])

heat_all = sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['natural-gas'])
heat_1 = sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['natural-gas'])
heat_2 = sum(model.Ip[i] * model.utin[i,m] for i in DEPOLYMERIZATION for m in ['natural-gas'])
heat_3 = sum(model.Ip[i] * model.utin[i,m] for i in UPGRADING for m in ['natural-gas'])

water_all = sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['cooling-water'])
water_1 = sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['cooling-water'])
water_2 = sum(model.Ip[i] * model.utin[i,m] for i in DEPOLYMERIZATION for m in ['cooling-water'])
water_3 = sum(model.Ip[i] * model.utin[i,m] for i in UPGRADING for m in ['cooling-water'])

### Annual revenue

In [71]:
# final products
revenue_final_product = sum(model.Fout[i] * process_rel[m][i] * price_p3.loc[m].item() for i in UPGRADING for m in FINALPRODUCT)

# coproducts
revenue_coproduct_glucose = sum(model.Foutfr[i,m] * price_by.loc[m].item() for i in FRACTIONATION for m in ['glucose'])
revenue_coproduct_xylose = sum(model.Foutfr[i,m] * price_by.loc[m].item() for i in FRACTIONATION for m in ['xylose','furfural'])
revenue_coproduct1 = revenue_coproduct_glucose + revenue_coproduct_xylose

revenue_coproduct_char = sum(model.Foutdpl[i,m] * price_by.loc[m].item() for i in DEPOLYMERIZATION for m in ['char'])
revenue_coproduct2 = revenue_coproduct_char

revenue_coproduct_gly = sum(model.Foutupg[i,m] * price_by.loc[m].item() for i in UPGRADING for m in ['glycerol'])
revenue_coproduct3 = revenue_coproduct_gly

In [72]:
# boiler/turbogenerator
revenue_BOTU = Fout_BOTU * price_ut.loc['electricity'].item()

In [73]:
revenue_coproduct = revenue_coproduct1 + revenue_coproduct2 + revenue_coproduct3
AR = revenue_final_product + revenue_coproduct + revenue_BOTU

### Depreciation

In [74]:
DP = [CAPEX/10]*10+[0]*20

### Objective function

In [75]:
EBITDA = (AR - OPEX - feedstock_cost)*days - CAPEX * 1000000/n

In [76]:
CF = [0]*n
NPV = - CAPEX*1000000*ft[0]+DP[0]
CF[0] = NPV

for t in range(1,n):
    CF[t] = - CAPEX*1000000*ft[t]+(AR - OPEX - feedstock_cost)*days*(1-tax) + DP[t]*tax
    NPV += CF[t]/((1+ir)**t)

In [77]:
model.objective = pe.Objective(expr = NPV, sense = pe.maximize)

## Result

In [78]:
model.pprint()

101 Set Declarations
    F_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                        : Size : Members
        None :     3 : F_index_0*F_index_1*F_index_2 : 6498 : {('milling', 'milling', 'hardwood'), ('milling', 'milling', 'softwood'), ('milling', 'milling', 'herbaceous-plant'), ('milling', 'milling', 'cellulose'), ('milling', 'milling', 'hemicellulose'), ('milling', 'milling', 'lignin1'), ('milling', 'milling', 'glucose'), ('milling', 'milling', 'xylose'), ('milling', 'milling', 'furfural'), ('milling', 'milling', 'lignin2'), ('milling', 'milling', 'lignin-monomers'), ('milling', 'milling', 'char'), ('milling', 'milling', 'unconverted-lignin'), ('milling', 'milling', 'PHA'), ('milling', 'milling', 'glycerol'), ('milling', 'milling', 'methyl-oleate'), ('milling', 'milling', 'adipic-acid'), ('milling', 'milling', 'PDC'), ('milling', 'DSA', 'hardwood'), ('milling', 'DSA', 'softwood'), ('milling', 'DSA', 'herbaceous-plant'), ('milling', 'DSA', 'cellulo

                                                                  ('BCD', 'AFEX', 'PHA') :     0 :     0 :  None : False : False : NonNegativeReals
                                                          ('BCD', 'AFEX', 'adipic-acid') :     0 :     0 :  None : False : False : NonNegativeReals
                                                            ('BCD', 'AFEX', 'cellulose') :     0 :     0 :  None : False : False : NonNegativeReals
                                                                 ('BCD', 'AFEX', 'char') :     0 :     0 :  None : False : False : NonNegativeReals
                                                             ('BCD', 'AFEX', 'furfural') :     0 :     0 :  None : False : False : NonNegativeReals
                                                              ('BCD', 'AFEX', 'glucose') :     0 :     0 :  None : False : False : NonNegativeReals
                                                             ('BCD', 'AFEX', 'glycerol') :     0 :     0 :  None

                                                ('GVL', 'organosolv', 'lignin-monomers') :     0 :     0 :  None : False : False : NonNegativeReals
                                                        ('GVL', 'organosolv', 'lignin1') :     0 :     0 :  None : False : False : NonNegativeReals
                                                        ('GVL', 'organosolv', 'lignin2') :     0 :     0 :  None : False : False : NonNegativeReals
                                                  ('GVL', 'organosolv', 'methyl-oleate') :     0 :     0 :  None : False : False : NonNegativeReals
                                                       ('GVL', 'organosolv', 'softwood') :     0 :     0 :  None : False : False : NonNegativeReals
                                             ('GVL', 'organosolv', 'unconverted-lignin') :     0 :     0 :  None : False : False : NonNegativeReals
                                                         ('GVL', 'organosolv', 'xylose') :     0 :     0 :  None

                                                  ('HyThUp', 'noprocess', 'adipic-acid') :     0 :     0 :  None : False : False : NonNegativeReals
                                                    ('HyThUp', 'noprocess', 'cellulose') :     0 :     0 :  None : False : False : NonNegativeReals
                                                         ('HyThUp', 'noprocess', 'char') :     0 :     0 :  None : False : False : NonNegativeReals
                                                     ('HyThUp', 'noprocess', 'furfural') :     0 :     0 :  None : False : False : NonNegativeReals
                                                      ('HyThUp', 'noprocess', 'glucose') :     0 :     0 :  None : False : False : NonNegativeReals
                                                     ('HyThUp', 'noprocess', 'glycerol') :     0 :     0 :  None : False : False : NonNegativeReals
                                                     ('HyThUp', 'noprocess', 'hardwood') :     0 :     0 :  None

                                    ('PDC-ferm', 'lipid-ferm_biodiesel-prod', 'lignin1') :     0 :     0 :  None : False : False : NonNegativeReals
                                    ('PDC-ferm', 'lipid-ferm_biodiesel-prod', 'lignin2') :     0 :     0 :  None : False : False : NonNegativeReals
                              ('PDC-ferm', 'lipid-ferm_biodiesel-prod', 'methyl-oleate') :     0 :     0 :  None : False : False : NonNegativeReals
                                   ('PDC-ferm', 'lipid-ferm_biodiesel-prod', 'softwood') :     0 :     0 :  None : False : False : NonNegativeReals
                         ('PDC-ferm', 'lipid-ferm_biodiesel-prod', 'unconverted-lignin') :     0 :     0 :  None : False : False : NonNegativeReals
                                     ('PDC-ferm', 'lipid-ferm_biodiesel-prod', 'xylose') :     0 :     0 :  None : False : False : NonNegativeReals
                                                          ('PDC-ferm', 'milling', 'PDC') :     0 :     0 :  None

...

        Key                             : Lower : Value : Upper : Fixed : Stale : Domain
                  ('AFEX', 'cellulose') :     0 :     0 :  None : False : False : NonNegativeReals
              ('AFEX', 'hemicellulose') :     0 :     0 :  None : False : False : NonNegativeReals
                    ('AFEX', 'lignin1') :     0 :     0 :  None : False : False : NonNegativeReals
                   ('DSA', 'cellulose') :     0 :     0 :  None : False : False : NonNegativeReals
               ('DSA', 'hemicellulose') :     0 :     0 :  None : False : False : NonNegativeReals
                     ('DSA', 'lignin1') :     0 :     0 :  None : False : False : NonNegativeReals
                   ('GVL', 'cellulose') :     0 :     0 :  None : False : False : NonNegativeReals
               ('GVL', 'hemicellulose') :     0 :     0 :  None : False : False : NonNegativeReals
                     ('GVL', 'lignin1') :     0 :     0 :  None : False : False : NonNegativeReals
                   (

        Key  : Active : Sense    : Expression
        None :   True : maximize : - 3.85*(41.44*sizefraction[milling]**0.6 + 145.38*sizefraction[DSA]**0.6 + 153.34*sizefraction[SEP]**0.6 + 144.26*sizefraction[LHW]**0.6 + 136.93*sizefraction[organosolv]**0.6 + 183.4*sizefraction[AFEX]**0.6 + 386.3*sizefraction[GVL]**0.6 + 58.19*sizefraction[alkaline]**0.6 + 16.93*sizefraction[BCD]**0.6 + 69.24*sizefraction[pyrolysis]**0.6 + 84.01*sizefraction[HDO]**0.6 + 94.17*sizefraction[HyThUp]**0.6 + 17.03*sizefraction[HYDRO]**0.6 + 27.59*sizefraction[PHA-ferm]**0.6 + 45.35013864966894*sizefraction[lipid-ferm_biodiesel-prod]**0.6 + 141.81*sizefraction[MA-ferm_AA-prod]**0.6 + 35.78*sizefraction[PDC-ferm]**0.6 + 41.59 + 90.63*((5.6*(Foutdpl[BCD,unconverted-lignin] + Foutdpl[pyrolysis,unconverted-lignin] + Foutdpl[HDO,unconverted-lignin] + Foutdpl[HyThUp,unconverted-lignin] + Foutdpl[HYDRO,unconverted-lignin] + Foutdpl[noprocess,unconverted-lignin] + Foutdpl[burn,unconverted-lignin]) + 5.6*(Fburnfr[DSA,

        Key                                                : Lower : Body                                                                                              : Upper : Active
                                            ('AFEX', 'AS') :   0.0 :                                                                                     chin[AFEX,AS] :   0.0 :   True
                                           ('AFEX', 'CSL') :   0.0 :                                                                                    chin[AFEX,CSL] :   0.0 :   True
                                           ('AFEX', 'DAP') :   0.0 :                                                                                    chin[AFEX,DAP] :   0.0 :   True
                                           ('AFEX', 'DSP') :   0.0 :                                                                                    chin[AFEX,DSP] :   0.0 :   True
                                          ('AFEX', 'EtOH') :   0.0 :            

        Key : Lower : Body                                                                                                                                                                                                                         : Upper : Active
          1 :  -Inf :                                                                                                                                                               F[BCD,PHA-ferm,lignin-monomers] - Foutdpl[BCD,lignin-monomers] :   0.0 :   True
          2 :   0.0 :                               Foutdpl[BCD,lignin-monomers] - (F[BCD,PHA-ferm,lignin-monomers] + F[BCD,lipid-ferm_biodiesel-prod,lignin-monomers] + F[BCD,MA-ferm_AA-prod,lignin-monomers] + F[BCD,PDC-ferm,lignin-monomers]) :   0.0 :   True
          3 :  -Inf :                                                                                                                                              F[BCD,lipid-ferm_biodiesel-prod,lignin-monomers] - Foutdp

        Key : Lower : Body                                                                                                                                                                                                                                               : Upper : Active
          1 :  -Inf :                                                                                                                                                                                                           F[DSA,BCD,lignin2] - Foutfr[DSA,lignin2] :   0.0 :   True
          2 :   0.0 :                                                         Foutfr[DSA,lignin2] - (F[DSA,BCD,lignin2] + F[DSA,pyrolysis,lignin2] + F[DSA,HDO,lignin2] + F[DSA,HyThUp,lignin2] + F[DSA,HYDRO,lignin2] + F[DSA,noprocess,lignin2] + F[DSA,burn,lignin2]) :   0.0 :   True
          3 :  -Inf :                                                                                                                                     

In [79]:
solver = pe.SolverFactory('baron')
solver.solve(model, options={'MaxIter': 200,
                            'EpsA':1e-12,
                            'AbsConFeasTol':1e-12,
                            'IsolTol':1e-12}, tee=True)

 BARON version 23.6.23. Built: OSX-64 Fri Jun 23 12:58:03 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Juliana Vasco-Correa at Penn State University, julianavasco@psu.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Kilinc, M. and N. V. Sahinidis, Exploiting integrality in the global
 optimization of mixed-integer nonlinear programming problems in BARON,
 Optimization Methods and Software, 33, 540-562, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSQP
 Doing local search
 Preprocessing found feasible solution with value -0.857022E+09
 Solving bounding LP
 Preprocessing found feasible solution with value 0.125900E+10
 Starting multi-start local search
 Done with local search
  Iteration    Open nodes         Time (s)    Lower bound      Upp

{'Problem': [{'Name': 'problem', 'Lower bound': 1258995378.58, 'Upper bound': 1258996637.58, 'Number of objectives': 1, 'Number of constraints': 1070, 'Number of variables': 743, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Error rc': 0, 'Time': 1.1354100704193115}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### solution

In [80]:
def print_solution(model):
    print("Variable Names\t\tValue")
    for c in model.component_data_objects(pe.Var):
        if c.value is None:
            print(c.name,"\t\t UNINITIALIZED")
        else:
            print(c.name,"\t\t", pe.value(c))
        
                
    print("\nObjective Name\t\tValue")
    for c in model.component_data_objects(pe.Objective):
        print(c.name,"\t\t", pe.value(c))

In [81]:
print_solution(model)

Variable Names		Value
Ifs[hardwood] 		 0.0
Ifs[softwood] 		 1.0
Ifs[herbaceous-plant] 		 0.0
Ip[milling] 		 1.0
Ip[DSA] 		 0.0
Ip[SEP] 		 0.0
Ip[LHW] 		 0.0
Ip[organosolv] 		 0.0
Ip[AFEX] 		 0.0
Ip[GVL] 		 0.0
Ip[alkaline] 		 1.0
Ip[BCD] 		 0.0
Ip[pyrolysis] 		 0.0
Ip[HDO] 		 0.0
Ip[HyThUp] 		 0.0
Ip[HYDRO] 		 0.0
Ip[noprocess] 		 1.0
Ip[burn] 		 0.0
Ip[PHA-ferm] 		 0.0
Ip[lipid-ferm_biodiesel-prod] 		 0.0
Ip[MA-ferm_AA-prod] 		 0.0
Ip[PDC-ferm] 		 1.0
Fin[milling] 		 2000.0
Fin[DSA] 		 0.0
Fin[SEP] 		 0.0
Fin[LHW] 		 0.0
Fin[organosolv] 		 0.0
Fin[AFEX] 		 0.0
Fin[GVL] 		 0.0
Fin[alkaline] 		 2000.0
Fin[BCD] 		 0.0
Fin[pyrolysis] 		 0.0
Fin[HDO] 		 0.0
Fin[HyThUp] 		 0.0
Fin[HYDRO] 		 0.0
Fin[noprocess] 		 282.0
Fin[burn] 		 0.0
Fin[PHA-ferm] 		 0.0
Fin[lipid-ferm_biodiesel-prod] 		 0.0
Fin[MA-ferm_AA-prod] 		 0.0
Fin[PDC-ferm] 		 282.0
Fout[milling] 		 2000.0
Fout[DSA] 		 0.0
Fout[SEP] 		 0.0
Fout[LHW] 		 0.0
Fout[organosolv] 		 0.0
Fout[AFEX] 		 0.0
Fout[GVL] 		 0.0
Fout[alkaline] 	

F[DSA,LHW,PDC] 		 0
F[DSA,organosolv,hardwood] 		 0
F[DSA,organosolv,softwood] 		 0
F[DSA,organosolv,herbaceous-plant] 		 0
F[DSA,organosolv,cellulose] 		 0
F[DSA,organosolv,hemicellulose] 		 0
F[DSA,organosolv,lignin1] 		 0
F[DSA,organosolv,glucose] 		 0
F[DSA,organosolv,xylose] 		 0
F[DSA,organosolv,furfural] 		 0
F[DSA,organosolv,lignin2] 		 0
F[DSA,organosolv,lignin-monomers] 		 0
F[DSA,organosolv,char] 		 0
F[DSA,organosolv,unconverted-lignin] 		 0
F[DSA,organosolv,PHA] 		 0
F[DSA,organosolv,glycerol] 		 0
F[DSA,organosolv,methyl-oleate] 		 0
F[DSA,organosolv,adipic-acid] 		 0
F[DSA,organosolv,PDC] 		 0
F[DSA,AFEX,hardwood] 		 0
F[DSA,AFEX,softwood] 		 0
F[DSA,AFEX,herbaceous-plant] 		 0
F[DSA,AFEX,cellulose] 		 0
F[DSA,AFEX,hemicellulose] 		 0
F[DSA,AFEX,lignin1] 		 0
F[DSA,AFEX,glucose] 		 0
F[DSA,AFEX,xylose] 		 0
F[DSA,AFEX,furfural] 		 0
F[DSA,AFEX,lignin2] 		 0
F[DSA,AFEX,lignin-monomers] 		 0
F[DSA,AFEX,char] 		 0
F[DSA,AFEX,unconverted-lignin] 		 0
F[DSA,AFEX,PHA] 		 0
F[D

F[SEP,LHW,PDC] 		 0
F[SEP,organosolv,hardwood] 		 0
F[SEP,organosolv,softwood] 		 0
F[SEP,organosolv,herbaceous-plant] 		 0
F[SEP,organosolv,cellulose] 		 0
F[SEP,organosolv,hemicellulose] 		 0
F[SEP,organosolv,lignin1] 		 0
F[SEP,organosolv,glucose] 		 0
F[SEP,organosolv,xylose] 		 0
F[SEP,organosolv,furfural] 		 0
F[SEP,organosolv,lignin2] 		 0
F[SEP,organosolv,lignin-monomers] 		 0
F[SEP,organosolv,char] 		 0
F[SEP,organosolv,unconverted-lignin] 		 0
F[SEP,organosolv,PHA] 		 0
F[SEP,organosolv,glycerol] 		 0
F[SEP,organosolv,methyl-oleate] 		 0
F[SEP,organosolv,adipic-acid] 		 0
F[SEP,organosolv,PDC] 		 0
F[SEP,AFEX,hardwood] 		 0
F[SEP,AFEX,softwood] 		 0
F[SEP,AFEX,herbaceous-plant] 		 0
F[SEP,AFEX,cellulose] 		 0
F[SEP,AFEX,hemicellulose] 		 0
F[SEP,AFEX,lignin1] 		 0
F[SEP,AFEX,glucose] 		 0
F[SEP,AFEX,xylose] 		 0
F[SEP,AFEX,furfural] 		 0
F[SEP,AFEX,lignin2] 		 0
F[SEP,AFEX,lignin-monomers] 		 0
F[SEP,AFEX,char] 		 0
F[SEP,AFEX,unconverted-lignin] 		 0
F[SEP,AFEX,PHA] 		 0
F[S

F[LHW,DSA,xylose] 		 0
F[LHW,DSA,furfural] 		 0
F[LHW,DSA,lignin2] 		 0
F[LHW,DSA,lignin-monomers] 		 0
F[LHW,DSA,char] 		 0
F[LHW,DSA,unconverted-lignin] 		 0
F[LHW,DSA,PHA] 		 0
F[LHW,DSA,glycerol] 		 0
F[LHW,DSA,methyl-oleate] 		 0
F[LHW,DSA,adipic-acid] 		 0
F[LHW,DSA,PDC] 		 0
F[LHW,SEP,hardwood] 		 0
F[LHW,SEP,softwood] 		 0
F[LHW,SEP,herbaceous-plant] 		 0
F[LHW,SEP,cellulose] 		 0
F[LHW,SEP,hemicellulose] 		 0
F[LHW,SEP,lignin1] 		 0
F[LHW,SEP,glucose] 		 0
F[LHW,SEP,xylose] 		 0
F[LHW,SEP,furfural] 		 0
F[LHW,SEP,lignin2] 		 0
F[LHW,SEP,lignin-monomers] 		 0
F[LHW,SEP,char] 		 0
F[LHW,SEP,unconverted-lignin] 		 0
F[LHW,SEP,PHA] 		 0
F[LHW,SEP,glycerol] 		 0
F[LHW,SEP,methyl-oleate] 		 0
F[LHW,SEP,adipic-acid] 		 0
F[LHW,SEP,PDC] 		 0
F[LHW,LHW,hardwood] 		 0
F[LHW,LHW,softwood] 		 0
F[LHW,LHW,herbaceous-plant] 		 0
F[LHW,LHW,cellulose] 		 0
F[LHW,LHW,hemicellulose] 		 0
F[LHW,LHW,lignin1] 		 0
F[LHW,LHW,glucose] 		 0
F[LHW,LHW,xylose] 		 0
F[LHW,LHW,furfural] 		 0
F[LHW,LHW,li

F[LHW,PDC-ferm,lignin-monomers] 		 0
F[LHW,PDC-ferm,char] 		 0
F[LHW,PDC-ferm,unconverted-lignin] 		 0
F[LHW,PDC-ferm,PHA] 		 0
F[LHW,PDC-ferm,glycerol] 		 0
F[LHW,PDC-ferm,methyl-oleate] 		 0
F[LHW,PDC-ferm,adipic-acid] 		 0
F[LHW,PDC-ferm,PDC] 		 0
F[organosolv,milling,hardwood] 		 0
F[organosolv,milling,softwood] 		 0
F[organosolv,milling,herbaceous-plant] 		 0
F[organosolv,milling,cellulose] 		 0
F[organosolv,milling,hemicellulose] 		 0
F[organosolv,milling,lignin1] 		 0
F[organosolv,milling,glucose] 		 0
F[organosolv,milling,xylose] 		 0
F[organosolv,milling,furfural] 		 0
F[organosolv,milling,lignin2] 		 0
F[organosolv,milling,lignin-monomers] 		 0
F[organosolv,milling,char] 		 0
F[organosolv,milling,unconverted-lignin] 		 0
F[organosolv,milling,PHA] 		 0
F[organosolv,milling,glycerol] 		 0
F[organosolv,milling,methyl-oleate] 		 0
F[organosolv,milling,adipic-acid] 		 0
F[organosolv,milling,PDC] 		 0
F[organosolv,DSA,hardwood] 		 0
F[organosolv,DSA,softwood] 		 0
F[organosolv,DSA,

...

F[MA-ferm_AA-prod,milling,PHA] 		 0
F[MA-ferm_AA-prod,milling,glycerol] 		 0
F[MA-ferm_AA-prod,milling,methyl-oleate] 		 0
F[MA-ferm_AA-prod,milling,adipic-acid] 		 0
F[MA-ferm_AA-prod,milling,PDC] 		 0
F[MA-ferm_AA-prod,DSA,hardwood] 		 0
F[MA-ferm_AA-prod,DSA,softwood] 		 0
F[MA-ferm_AA-prod,DSA,herbaceous-plant] 		 0
F[MA-ferm_AA-prod,DSA,cellulose] 		 0
F[MA-ferm_AA-prod,DSA,hemicellulose] 		 0
F[MA-ferm_AA-prod,DSA,lignin1] 		 0
F[MA-ferm_AA-prod,DSA,glucose] 		 0
F[MA-ferm_AA-prod,DSA,xylose] 		 0
F[MA-ferm_AA-prod,DSA,furfural] 		 0
F[MA-ferm_AA-prod,DSA,lignin2] 		 0
F[MA-ferm_AA-prod,DSA,lignin-monomers] 		 0
F[MA-ferm_AA-prod,DSA,char] 		 0
F[MA-ferm_AA-prod,DSA,unconverted-lignin] 		 0
F[MA-ferm_AA-prod,DSA,PHA] 		 0
F[MA-ferm_AA-prod,DSA,glycerol] 		 0
F[MA-ferm_AA-prod,DSA,methyl-oleate] 		 0
F[MA-ferm_AA-prod,DSA,adipic-acid] 		 0
F[MA-ferm_AA-prod,DSA,PDC] 		 0
F[MA-ferm_AA-prod,SEP,hardwood] 		 0
F[MA-ferm_AA-prod,SEP,softwood] 		 0
F[MA-ferm_AA-prod,SEP,herbaceous-plan

F[MA-ferm_AA-prod,noprocess,unconverted-lignin] 		 0
F[MA-ferm_AA-prod,noprocess,PHA] 		 0
F[MA-ferm_AA-prod,noprocess,glycerol] 		 0
F[MA-ferm_AA-prod,noprocess,methyl-oleate] 		 0
F[MA-ferm_AA-prod,noprocess,adipic-acid] 		 0
F[MA-ferm_AA-prod,noprocess,PDC] 		 0
F[MA-ferm_AA-prod,burn,hardwood] 		 0
F[MA-ferm_AA-prod,burn,softwood] 		 0
F[MA-ferm_AA-prod,burn,herbaceous-plant] 		 0
F[MA-ferm_AA-prod,burn,cellulose] 		 0
F[MA-ferm_AA-prod,burn,hemicellulose] 		 0
F[MA-ferm_AA-prod,burn,lignin1] 		 0
F[MA-ferm_AA-prod,burn,glucose] 		 0
F[MA-ferm_AA-prod,burn,xylose] 		 0
F[MA-ferm_AA-prod,burn,furfural] 		 0
F[MA-ferm_AA-prod,burn,lignin2] 		 0
F[MA-ferm_AA-prod,burn,lignin-monomers] 		 0
F[MA-ferm_AA-prod,burn,char] 		 0
F[MA-ferm_AA-prod,burn,unconverted-lignin] 		 0
F[MA-ferm_AA-prod,burn,PHA] 		 0
F[MA-ferm_AA-prod,burn,glycerol] 		 0
F[MA-ferm_AA-prod,burn,methyl-oleate] 		 0
F[MA-ferm_AA-prod,burn,adipic-acid] 		 0
F[MA-ferm_AA-prod,burn,PDC] 		 0
F[MA-ferm_AA-prod,PHA-ferm,har

F[PDC-ferm,BCD,hemicellulose] 		 0
F[PDC-ferm,BCD,lignin1] 		 0
F[PDC-ferm,BCD,glucose] 		 0
F[PDC-ferm,BCD,xylose] 		 0
F[PDC-ferm,BCD,furfural] 		 0
F[PDC-ferm,BCD,lignin2] 		 0
F[PDC-ferm,BCD,lignin-monomers] 		 0
F[PDC-ferm,BCD,char] 		 0
F[PDC-ferm,BCD,unconverted-lignin] 		 0
F[PDC-ferm,BCD,PHA] 		 0
F[PDC-ferm,BCD,glycerol] 		 0
F[PDC-ferm,BCD,methyl-oleate] 		 0
F[PDC-ferm,BCD,adipic-acid] 		 0
F[PDC-ferm,BCD,PDC] 		 0
F[PDC-ferm,pyrolysis,hardwood] 		 0
F[PDC-ferm,pyrolysis,softwood] 		 0
F[PDC-ferm,pyrolysis,herbaceous-plant] 		 0
F[PDC-ferm,pyrolysis,cellulose] 		 0
F[PDC-ferm,pyrolysis,hemicellulose] 		 0
F[PDC-ferm,pyrolysis,lignin1] 		 0
F[PDC-ferm,pyrolysis,glucose] 		 0
F[PDC-ferm,pyrolysis,xylose] 		 0
F[PDC-ferm,pyrolysis,furfural] 		 0
F[PDC-ferm,pyrolysis,lignin2] 		 0
F[PDC-ferm,pyrolysis,lignin-monomers] 		 0
F[PDC-ferm,pyrolysis,char] 		 0
F[PDC-ferm,pyrolysis,unconverted-lignin] 		 0
F[PDC-ferm,pyrolysis,PHA] 		 0
F[PDC-ferm,pyrolysis,glycerol] 		 0
F[PDC-ferm,p

Findpl[HYDRO,lignin2] 		 0.0
Findpl[noprocess,lignin2] 		 282.0
Findpl[burn,lignin2] 		 0.0
Foutdpl[BCD,lignin-monomers] 		 0.0
Foutdpl[BCD,char] 		 0.0
Foutdpl[BCD,unconverted-lignin] 		 0.0
Foutdpl[pyrolysis,lignin-monomers] 		 0.0
Foutdpl[pyrolysis,char] 		 0.0
Foutdpl[pyrolysis,unconverted-lignin] 		 0.0
Foutdpl[HDO,lignin-monomers] 		 0.0
Foutdpl[HDO,char] 		 0.0
Foutdpl[HDO,unconverted-lignin] 		 0.0
Foutdpl[HyThUp,lignin-monomers] 		 0.0
Foutdpl[HyThUp,char] 		 0.0
Foutdpl[HyThUp,unconverted-lignin] 		 0.0
Foutdpl[HYDRO,lignin-monomers] 		 0.0
Foutdpl[HYDRO,char] 		 0.0
Foutdpl[HYDRO,unconverted-lignin] 		 0.0
Foutdpl[noprocess,lignin-monomers] 		 282.0
Foutdpl[noprocess,char] 		 0.0
Foutdpl[noprocess,unconverted-lignin] 		 0.0
Foutdpl[burn,lignin-monomers] 		 0.0
Foutdpl[burn,char] 		 0.0
Foutdpl[burn,unconverted-lignin] 		 0.0
Finupg[PHA-ferm,lignin-monomers] 		 0.0
Finupg[lipid-ferm_biodiesel-prod,lignin-monomers] 		 0.0
Finupg[MA-ferm_AA-prod,lignin-monomers] 		 0.0
Finupg[P

chin[noprocess,chloroform] 		 0.0
chin[noprocess,H3PO4] 		 0.0
chin[noprocess,GVL] 		 0.0
chin[noprocess,glucose] 		 0.0
chin[noprocess,AS] 		 0.0
chin[noprocess,DSP] 		 0.0
chin[noprocess,MPP] 		 0.0
chin[noprocess,boiler_chem] 		 0.0
chin[noprocess,FGD] 		 0.0
chin[burn,water] 		 0.0
chin[burn,NaOH] 		 0.0
chin[burn,H2SO4] 		 0.0
chin[burn,NH3] 		 0.0
chin[burn,EtOH] 		 0.0
chin[burn,anthraquinone] 		 0.0
chin[burn,H2] 		 0.0
chin[burn,cellulase-enzymes] 		 0.0
chin[burn,CSL] 		 0.0
chin[burn,DAP] 		 0.0
chin[burn,methanol] 		 0.0
chin[burn,chloroform] 		 0.0
chin[burn,H3PO4] 		 0.0
chin[burn,GVL] 		 0.0
chin[burn,glucose] 		 0.0
chin[burn,AS] 		 0.0
chin[burn,DSP] 		 0.0
chin[burn,MPP] 		 0.0
chin[burn,boiler_chem] 		 0.0
chin[burn,FGD] 		 0.0
chin[PHA-ferm,water] 		 0.0
chin[PHA-ferm,NaOH] 		 0.0
chin[PHA-ferm,H2SO4] 		 0.0
chin[PHA-ferm,NH3] 		 0.0
chin[PHA-ferm,EtOH] 		 0.0
chin[PHA-ferm,anthraquinone] 		 0.0
chin[PHA-ferm,H2] 		 0.0
chin[PHA-ferm,cellulase-enzymes] 		 0.0
chin[P

In [82]:
print("net present value",f'{NPV():,}')

print("EBITDA",f'{EBITDA():,}')

print("annual cash flow",f'{AR()*days-OPEX()*days-feedstock_cost()*days:,}')

print("capital cost",f'{CAPEX()*1000000:,}')
print("capital cost (annulized with interest rate)",f'{CAPEX()*1000000* ir / (1-(1/(1+ir)**n)):,}')
print("annual operating cost",f'{OPEX()*days:,}')
print("annual chemicals cost",f'{chemical_cost()*days:,}')
print("annual utility cost",f'{utility_cost()*days:,}')
print("annual wastewater treatment cost",f'{Cw*days:,}')
print("annual feedstock cost",f'{feedstock_cost()*days:,}')
print("annual fixed operating cost",f'{FOC()*days:,}')
print("total annual cost",f'{CAPEX()*1000000* ir / (1-(1/(1+ir)**n))+OPEX()*days+feedstock_cost()*days:,}')
print("annual fractionation cost",f'{CAPEX1()*1000000* ir / (1-(1/(1+ir)**n))+OPEX1()*days:,}')
print("annual depolymerization cost",f'{CAPEX2()*1000000* ir / (1-(1/(1+ir)**n))+OPEX2()*days:,}')
print("annual upgrading cost",f'{CAPEX3()*1000000* ir / (1-(1/(1+ir)**n))+OPEX3()*days:,}')

print("total annual revenue",f'{AR()*days:,}')
print("annual revenue from fractionation",f'{revenue_coproduct1()*days:,}')
print("annual revenue from depolymerization",f'{revenue_coproduct2()*days:,}')    
print("annual revenue from final product",f'{revenue_final_product()*days:,}')

print("annual revenuet from glucose",f'{revenue_coproduct_glucose()*days:,}')
print("annual revenue from xylose",f'{revenue_coproduct_xylose()*days:,}')   
print("annual revenue from char",f'{revenue_coproduct_char()*days:,}') 
print("annual revenue from coproduct",f'{revenue_coproduct()*days:,}') 
print("annual revenue from boiler/turbogenerator",f'{revenue_BOTU()*days:,}') 

net present value 1,258,995,378.5829942
EBITDA 330,449,106.82892543
annual cash flow 413,085,612.96298903
capital cost 1,652,730,122.6812708
capital cost (annulized with interest rate) 194,129,060.06042558
annual operating cost 172,925,893.55701107
annual chemicals cost 76,773,158.70788291
annual utility cost 44,880,831.16869
annual wastewater treatment cost 1,690,000.0
annual feedstock cost 57,420,000.0
annual fixed operating cost 49,581,903.680438116
total annual cost 424,474,953.61743665
annual fractionation cost 280,817,078.5008637
annual depolymerization cost 247,849,086.0258637
annual upgrading cost 305,070,655.1095537
total annual revenue 643,431,506.5200001
annual revenue from fractionation 59,822,333.99999999
annual revenue from depolymerization 0.0
annual revenue from final product 455,528,700.0
annual revenuet from glucose 43,133,436.72
annual revenue from xylose 16,688,897.280000001
annual revenue from char 0.0
annual revenue from coproduct 59,822,333.99999999
annual revenu

In [83]:
cash_flow = []
for i in range(n):
    cash_flow.append(CF[i]())
    
# Calculate the IRR
irr = npf.irr(cash_flow)

# Print the IRR as a percentage
print(f"The IRR is: {irr * 100:.2f}%")

The IRR is: 25.63%


### Utility usage

In [84]:
# annual utility usage
# UTILITY =['cooling-water','electricity','natural-gas']
electrcity_all = days*sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['electricity'])
electrcity_milling = days* sum(model.Ip[i] * model.utin[i,m] for i in MILLING for m in ['electricity'])
electrcity_1 = days* sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['electricity'])
electrcity_2 = days*sum(model.Ip[i] * model.utin[i,m] for i in DEPOLYMERIZATION for m in ['electricity'])
electrcity_3 = days*sum(model.Ip[i] * model.utin[i,m] for i in UPGRADING for m in ['electricity'])

heat_all = days*sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['natural-gas'])
heat_milling = days*sum(model.Ip[i] * model.utin[i,m] for i in MILLING for m in ['natural-gas'])
heat_1 = days*sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['natural-gas'])
heat_2 = days*sum(model.Ip[i] * model.utin[i,m] for i in DEPOLYMERIZATION for m in ['natural-gas'])
heat_3 = days*sum(model.Ip[i] * model.utin[i,m] for i in UPGRADING for m in ['natural-gas'])

water_all = days*sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['cooling-water'])
water_milling = days*sum(model.Ip[i] * model.utin[i,m] for i in MILLING for m in ['cooling-water'])
water_1 = days*sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['cooling-water'])
water_2 = days*sum(model.Ip[i] * model.utin[i,m] for i in DEPOLYMERIZATION for m in ['cooling-water'])
water_3 = days*sum(model.Ip[i] * model.utin[i,m] for i in UPGRADING for m in ['cooling-water'])

In [85]:
print(f'annual electrcity usage is {electrcity_all()/1000:,} mwh')
print(f'annual electrcity usage in milling is {electrcity_milling()/1000:,} mwh')
print(f'annual electrcity usage in fractionation is {electrcity_1()/1000:,} mwh')
print(f'annual electrcity usage in depolymerization is {electrcity_2()/1000:,} mwh')
print(f'annual electrcity usage in upgrading is {electrcity_3()/1000:,} mwh')

annual electrcity usage is 217,402.91099999996 mwh
annual electrcity usage in milling is 132,000.0 mwh
annual electrcity usage in fractionation is 778.8 mwh
annual electrcity usage in depolymerization is 0.0 mwh
annual electrcity usage in upgrading is 84,624.111 mwh


In [86]:
print(f'annual natural gas usage is {heat_all()/1000:,} mwh')
print(f'annual natural gas usage in milling is {heat_milling()/1000:,} mwh')
print(f'annual natural gas usage in fractionation is {heat_1()/1000:,} mwh')
print(f'annual natural gas usage in depolymerization is {heat_2()/1000:,} mwh')
print(f'annual natural gas usage in upgrading is {heat_3()/1000:,} mwh')

annual natural gas usage is 737,949.0624 mwh
annual natural gas usage in milling is 0.0 mwh
annual natural gas usage in fractionation is 190,845.6 mwh
annual natural gas usage in depolymerization is 0.0 mwh
annual natural gas usage in upgrading is 547,103.4624 mwh


In [87]:
print(f'annual cooling water usage is {water_all()/1000:,} mwh')
print(f'annual cooling water usage in milling is {water_milling()/1000:,} mwh')
print(f'annual cooling water usage in fractionation is {water_1()/1000:,} mwh')
print(f'annual cooling water usage in depolymerization is {water_2()/1000:,} mwh')
print(f'annual cooling water usage in upgrading is {water_3()/1000:,} mwh')

annual cooling water usage is 424,718.3819999999 mwh
annual cooling water usage in milling is 0.0 mwh
annual cooling water usage in fractionation is 66,000.0 mwh
annual cooling water usage in depolymerization is 0.0 mwh
annual cooling water usage in upgrading is 358,718.3819999999 mwh
